In [3]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。
与えられた売上データを分析するために、以下の主要な視点から傾向を探ります。

### 1. カテゴリー別売上分析
各カテゴリー（食品、衣服、日用品）ごとに合計売上を算出し、それぞれのカテゴリーが総売上に占める割合を確認します。これにより、どのカテゴリーが最も収益を上げているかがわかります。

### 2. 売上日別の傾向
売上日ごとのトレンドを視覚化することで、売上が多い日や季節的な変動があるかどうかを調べます。特定の日に売上が急増している場合、その要因を分析することが重要です。

### 3. 商品別売上分析
各商品の売上高、数量、利益を見積もり、どの商品が最も利益を上げているかを特定します